In [1]:
import yaml
import pickle
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import matplotlib as mpl
from astropy import constants, units

from scripts import *

plt.rcParams['font.size'] = 16
path = '/Users/arcticfox/Documents/GitHub/disks_and_outbursts/radmc'
path = '/Users/arcticfox/Documents/disks'

## MIST table

In [ ]:
model_path = '/Users/arcticfox/Documents/disks/MIST_models'
model_fns = np.sort([os.path.join(model_path, i) for i in os.listdir(model_path)])

In [ ]:
for i in range(len(model_fns)):
    tab = Table.read(model_fns[i], format='ascii')
    row = tab[tab['star_age']>=10**6][0]
    print(row['star_mass'], 10**row['log_Teff'], 10**row['log_L'])

In [ ]:
steps = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]

## RADMC model call

In [ ]:
norm = setup(PATH=path, models=['bigdisk'], 
             uv=True, run=True, ndust=2,
             disk_params={'r_in':0.1, 'r_out':50.0, 'r_peb':50.0, 'r_snow':0.},
             grid_params={'ntheta':90, 
                          'nr':110, 
                          'n_in':0.05},
             star_params={'L_uv_star':10**3},
             disk_type='herbig', nphot=10**7)
            ## nr = 100; ntheta=96

bigdisk
saved star spectrum
0.1 50.0
Saved radmc setup files in /Users/arcticfox/Documents/disks/models/bigdisk


In [ ]:
norm = setup(PATH=path, models=['bigdisk100'], 
             uv=True, run=True,
             disk_params={'r_in':0.1, 'r_out':50.0, 'r_peb':50.0, 'r_snow':0},
             grid_params={'ntheta':100, 
                          'nr':120, 
                          'n_in':0.05},
             star_params={'T_star':6000,
                          'L_uv_star':10**6},
             disk_type='herbig')
            ## nr = 100; ntheta=96

In [ ]:
def open_pickles(path):
    with open(os.path.join(path,'diskdata.pkl'), 'rb') as infile:
              diskinp = pickle.load(infile)
              ddustsm, ddustlg, tdustsm, tdustlg, dgas, tgas, re, ze = diskinp
    return ddustsm, ddustlg, tdustsm, tdustlg, dgas, tgas, re, ze

In [ ]:
bigdisk = open_pickles('/Users/arcticfox/Documents/disks/models/bigdisk')
bigdisk100 = open_pickles('/Users/arcticfox/Documents/disks/models/bigdisk100')       

In [ ]:
12000**4/6000**4

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(14,6))

dat100 = np.rot90(bigdisk100[3]+bigdisk100[2],3)
tdust_ticks = np.linspace(30,500,9)
im = ax2.imshow(dat100, origin='lower',
           norm= mpl.colors.LogNorm(vmin=np.nanmin(tdust_ticks), 
                                    vmax=np.nanmax(tdust_ticks)),
           cmap='RdYlBu_r')
ax2.set_title('L=100Lsun')

dat = np.rot90(bigdisk[3]+bigdisk[2],3)
ax1.imshow(dat, origin='lower',
           norm= mpl.colors.LogNorm(vmin=np.nanmin(tdust_ticks), 
                                    vmax=np.nanmax(tdust_ticks)),
           cmap='RdYlBu_r')

ax3.imshow(dat100-dat, origin='lower',
           norm= mpl.colors.LogNorm(vmin=np.nanmin(tdust_ticks), 
                                    vmax=np.nanmax(tdust_ticks)),
           cmap='RdYlBu_r')

In [ ]:
star100 = np.loadtxt('/Users/arcticfox/Documents/disks/models/bigdisk100/starspec.txt')
star = np.loadtxt('/Users/arcticfox/Documents/disks/models/bigdisk/starspec.txt')

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(star100[0], star100[1], label='100x luminosity')
plt.plot(star[0],star[1], label='normal luminosity')
plt.legend()
plt.yscale('log')
plt.xscale('log')

In [ ]:
((6000*3)/5777)**4